### WEEK 3 ASSESSMENT

In [214]:
# Import packages
import os

import pandas as pd
import numpy as np

from sklearn.linear_model import ElasticNetCV, ElasticNet

from matplotlib import pyplot as plt
%matplotlib inline

In [215]:
############################### FUNCTIONS ############################

#import helper functions
from fn_helpers import fn_print_coefficients, fn_plot_predictions

#creates linear splines of a variable
def fn_tosplines(varname,x):
#takes as input a feature name and the values of that feature
    
    
    #split feature into percentiles 
    ptiles = np.percentile(x,5*(1+np.arange(20)))
#     print(ptiles)

    #create dataframe which contains the original variable and its original values
    df_ptiles = pd.DataFrame({var: x})

    #now add splines
    for idx, ptile in enumerate(ptiles):
        #NOTE: spline variable is max(0, x - value of knot)
        df_ptiles[var + '_' + str(idx)] = np.maximum(0, x - ptiles[idx])
    return [df_ptiles, ptiles]

def fn_MAE(actuals, predictions):
#     return np.round(np.mean(np.abs(predictions - actuals)))
    return (np.mean(np.abs(predictions - actuals)))

Starting from importing the Ames house price data with pandas (read csv)

In [216]:
# Set directories and paths
print(os.getcwd())
dirRawData = "../input/"
dirPData   = "../PData/"
dirPOutput = "../POutput/"

/home/jovyan/Projects/AmesHousing/PCode


In [217]:
############################ Load and read data ######################
# Using the dictionary below (ames_dtypes), use the read_csv function in Pandas to read AmesHousing.txt
# The name of the dataframe you create should be df_all
# Inspect the top few lines to make sure they look sensible
# Note that the column names contain . to separate words and they contain capitals, e.g. Lot.Frontage. Change all column names so that capitals are replaced with lower case and periods (.) are replaced with underscores (_). One way to do this is
# Use sub from the re package to replace . with underscore (_) in all column names
# Use the .lower method on each string to change all letters to lower case

ames_dtypes = {'Order': np.int64,
             'PID': np.int64,
             'MS.SubClass': np.object,
             'MS.Zoning': np.object,
             'Lot.Frontage': np.float64,
             'Lot.Area': np.float64,
             'Street': np.object, 
             'Alley': np.object, 
             'Lot.Shape': np.object, 
             'Land.Contour': np.object,  
             'Lot.Config': np.object,
             'Land.Slope': np.object,
             'Neighborhood': np.object,
             'Condition.1': np.object,
             'Condition.2': np.object,
             'Bldg.Type': np.object,
             'House.Style': np.object,
             'Overall.Qual': np.float64,
             'Overall.Cond': np.float64,
             'Year.Built': np.float64,
             'Year.Remod.Add': np.float64,
             'Roof.Style': np.object,
             'Roof.Matl': np.object,
             'Exterior.1st': np.object,
             'Exterior.2nd': np.object,
             'Mas.Vnr.Type': np.object,
             'Mas.Vnr.Area': np.float64,
             'Exter.Qual': np.object,
             'Exter.Cond': np.object,
             'Foundation': np.object, 
             'Bsmt.Qual': np.object,
             'Bsmt.Cond': np.object,
             'Bsmt.Exposure': np.object,
             'BsmtFin.Type.1': np.object,
             'BsmtFin.SF.1': np.float64,
             'BsmtFin.Type.2': np.object,
             'BsmtFin.SF.2': np.float64,
             'Bsmt.Unf.SF': np.float64,
             'Total.Bsmt.SF': np.float64,
             'Heating': np.object,
             'Heating.QC': np.object,
             'Central.Air': np.object,
             'Electrical': np.object,
             'X1st.Flr.SF': np.float64,
             'X2nd.Flr.SF': np.float64,
             'Low.Qual.Fin.SF': np.float64,
             'Gr.Liv.Area': np.float64, 
             'Bsmt.Full.Bath': np.float64,
             'Bsmt.Half.Bath': np.float64,
             'Full.Bath': np.float64,
             'Half.Bath': np.float64,
             'Bedroom.AbvGr': np.float64,
             'Kitchen.AbvGr': np.float64,
             'Kitchen.Qual': np.object,
             'TotRms.AbvGrd': np.float64, 
             'Functional': np.object, 
             'Fireplaces': np.float64, 
             'Fireplace.Qu': np.object,
             'Garage.Type': np.object, 
             'Garage.Yr.Blt': np.float64,
             'Garage.Finish': np.object,
             'Garage.Cars': np.float64,
             'Garage.Area': np.float64, 
             'Garage.Qual': np.object, 
             'Garage.Cond': np.object, 
             'Paved.Drive': np.object,
             'Wood.Deck.SF': np.float64,
             'Open.Porch.SF': np.float64,
             'Enclosed.Porch': np.float64, 
             'X3Ssn.Porch': np.float64,
             'Screen.Porch': np.float64,
             'Pool.Area': np.float64,
             'Fence': np.object,
             'Misc.Feature': np.object,
             'Misc.Val': np.float64,
             'Mo.Sold': np.float64, 
             'Yr.Sold': np.float64,
             'Sale.Type': np.object,
             'Sale.Condition': np.object,
             'SalePrice': np.float64}
    

#default seperator is comma, but data is seperated with spaces so change sep, set dtype as prev. defined dict
df_all = pd.read_csv(dirRawData+'AmesHousing.txt',sep=' ', dtype = ames_dtypes, na_values = 'NA')
df_all.head() #inspect at first few rows

,Order,PID,MS.SubClass,MS.Zoning,Lot.Frontage,Lot.Area,Street,Alley,Lot.Shape,Land.Contour,...,Screen.Porch,Pool.Area,Fence,Misc.Feature,Misc.Val,Mo.Sold,Yr.Sold,Sale.Type,Sale.Condition,SalePrice
0,1,526301100,MS_20,RL,141.0,31770.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,5.0,2010.0,WD,Normal,215000.0
1,2,526350040,MS_20,RH,80.0,11622.0,Pave,DoesNotHaveOne,Reg,Lvl,...,120.0,0.0,MnPrv,DoesNotHaveOne,0.0,6.0,2010.0,WD,Normal,105000.0
2,3,526351010,MS_20,RL,81.0,14267.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,12500.0,6.0,2010.0,WD,Normal,172000.0
3,4,526353030,MS_20,RL,93.0,11160.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2010.0,WD,Normal,244000.0
4,5,527105010,MS_60,RL,74.0,13830.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,MnPrv,DoesNotHaveOne,0.0,3.0,2010.0,WD,Normal,189900.0


In [218]:
# Note that the column names contain . to separate words and they contain capitals, e.g. Lot.Frontage. 
# Change all column names so that capitals are replaced with lower case and periods (.) are replaced with 
# underscores (_). One way to do this is:
# Use str.replace from the pandas package to replace . with underscore (_) in all column names
# Use the .lower method on each string to change all letters to lower case


#df_all.columns is an index, diff to work with so let's work with values of index
columns = df_all.columns.values
#write a function to replace . with _ and convert string chars to lowercase
# test = "This.Is.A.Test" # to test function
def convert(colname):
    #replace '.' with '_'
    colname = colname.replace('.','_')
    colname = colname.lower() #change to lower case
    return colname
#map is an iterator and returns values one at a time, so must force it into a list
columns=list(map(convert, columns))
df_all.columns = columns #assign df_all.columns to revised columns
df_all.tail(10) #inspect first few rows

,order,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
2920,2921,923228310,MS_160,RM,21.0,1894.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2006.0,WD,Abnorml,71000.0
2921,2922,923229110,MS_90,RL,55.0,12640.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,7.0,2006.0,WD,Normal,150900.0
2922,2923,923230040,MS_90,RL,63.0,9297.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,7.0,2006.0,WD,Family,188000.0
2923,2924,923250060,MS_20,RL,80.0,17400.0,Pave,DoesNotHaveOne,Reg,Low,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,5.0,2006.0,WD,Normal,160000.0
2924,2925,923251180,MS_20,RL,160.0,20000.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,9.0,2006.0,WD,Abnorml,131000.0
2925,2926,923275080,MS_80,RL,37.0,7937.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,GdPrv,DoesNotHaveOne,0.0,3.0,2006.0,WD,Normal,142500.0
2926,2927,923276100,MS_20,RL,NaN,8885.0,Pave,DoesNotHaveOne,IR1,Low,...,0.0,0.0,MnPrv,DoesNotHaveOne,0.0,6.0,2006.0,WD,Normal,131000.0
2927,2928,923400125,MS_85,RL,62.0,10441.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,MnPrv,Shed,700.0,7.0,2006.0,WD,Normal,132000.0
2928,2929,924100070,MS_20,RL,77.0,10010.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2006.0,WD,Normal,170000.0
2929,2930,924151050,MS_60,RL,74.0,9627.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,11.0,2006.0,WD,Normal,188000.0


In [219]:
######################### DATA MANIPULATION ##########################
## declare variables
vars_all = df_all.columns.values
# df_all.shape #double check number of rows matches what expected
var_dep = ['saleprice']
vars_notToUse = ['order','pid']
#create list of independent variables
vars_ind = [var for var in vars_all if var not in (vars_notToUse+var_dep)]
#create list of numeric independent variables
vars_ind_numeric = [var for var in vars_ind if df_all[var].dtype != 'object']
#create list of categorical (independent) variables
vars_ind_categorical = [var for var in vars_ind if df_all[var].dtype == 'object']
# print(len(vars_ind_categorical))
           


In [220]:
################# Deal with missings and outliers ####################
# Note: should have exactly 2922 examples, test with df_all.shape

In [221]:
print(df_all.shape)
#collapse axis = 0 i.e. sum missing values,
#store as series
srs_missing = df_all.isnull().sum(axis=0) 
print(srs_missing[srs_missing>0]) #show which features have missing values

(2930, 80)
lot_frontage      490
mas_vnr_area       23
bsmtfin_sf_1        1
bsmtfin_sf_2        1
bsmt_unf_sf         1
total_bsmt_sf       1
bsmt_full_bath      2
bsmt_half_bath      2
garage_yr_blt     159
garage_cars         1
garage_area         1
dtype: int64


In [222]:
### REPLACE NaNs with non-NaN means if many missing values

In [223]:
# if only a few missing, drop examples with missing values

In [224]:
#store in list the variables which have more than 20 NaN values
vars_to_replace = [var for var in df_all if df_all[var].isnull().sum(axis=0) > 20]

In [225]:
#calculate their NaN mean (mean excluding NaNs)
means_to_replace = [np.nanmean(df_all[var].values) for var in vars_to_replace]

In [226]:
print(vars_to_replace)
print(means_to_replace)


['lot_frontage', 'mas_vnr_area', 'garage_yr_blt']
[69.22459016393442, 101.8968008255934, 1978.1324431613136]


In [227]:
###REPLACE missing values of vars_to_replace with the mean of the non missing values

In [228]:
df_temp=df_all[vars_to_replace].copy()
for var in vars_to_replace:
    i=0
    temp_nanmean = means_to_replace[vars_to_replace.index(var)]
    while i<len(df_temp[var]):
        if np.isnan(df_temp[var].iloc[i]):
            #then replace with mean of non-missing values
            df_temp[var].iloc[i]=temp_nanmean
        i+=1
df_all[vars_to_replace] = df_temp
# df_all.head(100) #just to double check NaNs have been changed

In [229]:
print(df_all.shape)
#collapse axis = 0 i.e. sum missing values,
#store as series
srs_missing = df_all.isnull().sum(axis=0) 
print(srs_missing[srs_missing>0]) #show which features have missing values

(2930, 80)
bsmtfin_sf_1      1
bsmtfin_sf_2      1
bsmt_unf_sf       1
total_bsmt_sf     1
bsmt_full_bath    2
bsmt_half_bath    2
garage_cars       1
garage_area       1
dtype: int64


In [230]:
### Now drop the examples with few missing values

In [231]:
df_all.dropna(axis=0, how='any', inplace=True)
srs_missing = df_all.isnull().sum(axis=0) 
print(srs_missing[srs_missing>0]) #double check no more nans

Series([], dtype: int64)


In [232]:
### Remove known outliers

In [233]:
#i.e. Remove the 5 examples where gr_liv_area > 4000 and check the number of rows again.

In [234]:
df_all = df_all[df_all['gr_liv_area']<=4000]

In [235]:
df_all.shape

(2922, 80)

In [236]:
df_all.tail(10)

,order,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
2920,2921,923228310,MS_160,RM,21.00000,1894.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2006.0,WD,Abnorml,71000.0
2921,2922,923229110,MS_90,RL,55.00000,12640.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,7.0,2006.0,WD,Normal,150900.0
2922,2923,923230040,MS_90,RL,63.00000,9297.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,7.0,2006.0,WD,Family,188000.0
2923,2924,923250060,MS_20,RL,80.00000,17400.0,Pave,DoesNotHaveOne,Reg,Low,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,5.0,2006.0,WD,Normal,160000.0
2924,2925,923251180,MS_20,RL,160.00000,20000.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,9.0,2006.0,WD,Abnorml,131000.0
2925,2926,923275080,MS_80,RL,37.00000,7937.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,GdPrv,DoesNotHaveOne,0.0,3.0,2006.0,WD,Normal,142500.0
2926,2927,923276100,MS_20,RL,69.22459,8885.0,Pave,DoesNotHaveOne,IR1,Low,...,0.0,0.0,MnPrv,DoesNotHaveOne,0.0,6.0,2006.0,WD,Normal,131000.0
2927,2928,923400125,MS_85,RL,62.00000,10441.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,MnPrv,Shed,700.0,7.0,2006.0,WD,Normal,132000.0
2928,2929,924100070,MS_20,RL,77.00000,10010.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2006.0,WD,Normal,170000.0
2929,2930,924151050,MS_60,RL,74.00000,9627.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,11.0,2006.0,WD,Normal,188000.0


In [237]:
## NOTE: Having dropped examples, must reset index to spline will not work
df_all.reset_index(drop=True, inplace=True)
df_all.tail(10)

,order,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,screen_porch,pool_area,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice
2912,2921,923228310,MS_160,RM,21.00000,1894.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2006.0,WD,Abnorml,71000.0
2913,2922,923229110,MS_90,RL,55.00000,12640.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,7.0,2006.0,WD,Normal,150900.0
2914,2923,923230040,MS_90,RL,63.00000,9297.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,7.0,2006.0,WD,Family,188000.0
2915,2924,923250060,MS_20,RL,80.00000,17400.0,Pave,DoesNotHaveOne,Reg,Low,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,5.0,2006.0,WD,Normal,160000.0
2916,2925,923251180,MS_20,RL,160.00000,20000.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,9.0,2006.0,WD,Abnorml,131000.0
2917,2926,923275080,MS_80,RL,37.00000,7937.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,0.0,GdPrv,DoesNotHaveOne,0.0,3.0,2006.0,WD,Normal,142500.0
2918,2927,923276100,MS_20,RL,69.22459,8885.0,Pave,DoesNotHaveOne,IR1,Low,...,0.0,0.0,MnPrv,DoesNotHaveOne,0.0,6.0,2006.0,WD,Normal,131000.0
2919,2928,923400125,MS_85,RL,62.00000,10441.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,MnPrv,Shed,700.0,7.0,2006.0,WD,Normal,132000.0
2920,2929,924100070,MS_20,RL,77.00000,10010.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2006.0,WD,Normal,170000.0
2921,2930,924151050,MS_60,RL,74.00000,9627.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,11.0,2006.0,WD,Normal,188000.0


In [238]:
######################## CREATE FOLD VARIABLE ########################

#create instance of random num generator with seed 2018
rng = np.random.RandomState(seed=2018)
# randint(low, high=None, size=None, dtype='l')
#generate as many random ints (in range [0,10]) as there are examples
fold = rng.randint(0,10, len(df_all))
#append to df_all
df_all['fold'] = fold
# df_all_onehot['fold'] = fold
df_all.head(10)

,order,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,alley,lot_shape,land_contour,...,pool_area,fence,misc_feature,misc_val,mo_sold,yr_sold,sale_type,sale_condition,saleprice,fold
0,1,526301100,MS_20,RL,141.0,31770.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,5.0,2010.0,WD,Normal,215000.0,6
1,2,526350040,MS_20,RH,80.0,11622.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,MnPrv,DoesNotHaveOne,0.0,6.0,2010.0,WD,Normal,105000.0,2
2,3,526351010,MS_20,RL,81.0,14267.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,DoesNotHaveOne,DoesNotHaveOne,12500.0,6.0,2010.0,WD,Normal,172000.0,9
3,4,526353030,MS_20,RL,93.0,11160.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2010.0,WD,Normal,244000.0,5
4,5,527105010,MS_60,RL,74.0,13830.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,MnPrv,DoesNotHaveOne,0.0,3.0,2010.0,WD,Normal,189900.0,4
5,6,527105030,MS_60,RL,78.0,9978.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,6.0,2010.0,WD,Normal,195500.0,6
6,7,527127150,MS_120,RL,41.0,4920.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,4.0,2010.0,WD,Normal,213500.0,9
7,8,527145080,MS_120,RL,43.0,5005.0,Pave,DoesNotHaveOne,IR1,HLS,...,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,1.0,2010.0,WD,Normal,191500.0,9
8,9,527146030,MS_120,RL,39.0,5389.0,Pave,DoesNotHaveOne,IR1,Lvl,...,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,3.0,2010.0,WD,Normal,236500.0,7
9,10,527162130,MS_60,RL,60.0,7500.0,Pave,DoesNotHaveOne,Reg,Lvl,...,0.0,DoesNotHaveOne,DoesNotHaveOne,0.0,6.0,2010.0,WD,Normal,189000.0,9


In [239]:
### Separate train and test
idx_train  = np.where(df_all['fold'].isin(np.arange(0,6)))[0]
idx_val    = np.where(df_all['fold'].isin([6,7]))[0]
idx_design = np.where(df_all['fold'].isin(np.arange(0,8)))[0]
idx_test   = np.where(df_all['fold'].isin([8,9]))[0]
print(idx_train.shape)
print(idx_design.shape)
print(idx_test.shape)

# idx_test

(1749,)
(2335,)
(587,)


In [240]:
################### ONE-HOT CATEGORICAL VARS ########################

In [241]:
# df_all[vars_ind_categorical].nunique() #show cardinality of each categorical variable
#cardinality not too high so can simply one-hot all cat variables

#start with empty list
vars_ind_onehot = []

#create copy to work with
df_all_onehot = df_all.copy()

for col in vars_ind_categorical:
#     print(col)
    
    # use pd.get_dummies on  df_all[col] 
    df_oh = pd.get_dummies(df_all[col], drop_first=False)
    
    # Find the column name of the most frequent category
    col_mostFreq =  df_oh.sum(axis=0).idxmax() 
    
    # Drop the column of the most frequent category
    df_oh = df_oh.drop(col_mostFreq, axis=1, inplace=False)
        
    # Rename the columns to have the original variable name as a prefix
    oh_names = col+'_'+df_oh.columns
    df_oh.columns = oh_names
    
    df_all_onehot = pd.concat([df_all_onehot, df_oh], axis = 1, sort = False)

    del df_all_onehot[col]
    vars_ind_onehot.extend(oh_names)
df_all_onehot.head()

,order,pid,lot_frontage,lot_area,overall_qual,overall_cond,year_built,year_remod_add,mas_vnr_area,bsmtfin_sf_1,...,misc_feature_Shed,sale_type_COD,sale_type_CWD,sale_type_ConLD,sale_type_New,sale_condition_Abnorml,sale_condition_AdjLand,sale_condition_Alloca,sale_condition_Family,sale_condition_Partial
0,1,526301100,141.0,31770.0,6.0,5.0,1960.0,1960.0,112.0,639.0,...,0,0,0,0,0,0,0,0,0,0
1,2,526350040,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,...,0,0,0,0,0,0,0,0,0,0
2,3,526351010,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,...,0,0,0,0,0,0,0,0,0,0
3,4,526353030,93.0,11160.0,7.0,5.0,1968.0,1968.0,0.0,1065.0,...,0,0,0,0,0,0,0,0,0,0
4,5,527105010,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,...,0,0,0,0,0,0,0,0,0,0


In [242]:
# vars_ind_onehot

In [243]:
#################### ADDING SPLINED FEATURES #########################

In [244]:
# For at least some of the numerical ind variables, add linear spline features

In [245]:
###### CONSIDER NOT USING ALL NUMERICAL IND VARIABLES


In [246]:
####EITHER EXCLUDE VARS CHOSEN BY PROF
# vars_toUse = vars_ind_numeric
# temp = ['bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf',
#         'bsmt_full_bath', 'bsmt_half_bath',
#         'garage_cars',
#         'bedroom_abvgr', 'kitchen_abvgr',
#         'full_bath', 'half_bath',
#         'x1st_flr_sf', 'x2nd_flr_sf', 'low_qual_fin_sf',
#         'total_bsmt_sf',
#         'totrms_abvgrd', 'lot_area', 'overall_qual']
# vars_toUse = [var for var in vars_toUse if var not in temp]
# X = df_all[vars_toUse].values
# y = df_all[var_dep].values

In [247]:
### ALTERNATIVELY CHOSE VARS TO EXCLUDE BY CARDINALITY

In [248]:
# Note that vars with less than 20 unique values will not spline well,
# A solution is not to spline those with cardinality < 20

###check cardinality
# print(df_all[vars_ind_numeric].nunique())
vars_ind_num_exclude = [var for var in vars_ind_numeric if len(df_all[var].unique())<20]

##vars_ind and remove those to be excluded
# print(vars_ind_num_exclude)
vars_ind_num_include = [var for var in vars_ind_numeric if var not in vars_ind_num_exclude]
print(vars_ind_num_include)
# df_all[vars_ind_num_include].nunique()
vars_toUse = vars_ind_num_include

['lot_frontage', 'lot_area', 'year_built', 'year_remod_add', 'mas_vnr_area', 'bsmtfin_sf_1', 'bsmtfin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'x1st_flr_sf', 'x2nd_flr_sf', 'low_qual_fin_sf', 'gr_liv_area', 'garage_yr_blt', 'garage_area', 'wood_deck_sf', 'open_porch_sf', 'enclosed_porch', 'x3ssn_porch', 'screen_porch', 'misc_val']


In [249]:
# len(vars_ind_num_exclude)

In [250]:
df_temp = df_all_onehot.copy()
print(df_all_onehot.shape)
print(df_temp.shape)
srs_missing = df_temp.isnull().sum(axis=0) 
print(srs_missing[srs_missing>0]) #double check no more nans

(2922, 223)
(2922, 223)
Series([], dtype: int64)


In [251]:
# ## Standardise BEFORE SPLINING

# ### Separate train and test
# idx_train  = np.where(df_all['fold'].isin(np.arange(0,6)))[0]
# idx_val    = np.where(df_all['fold'].isin([6,7]))[0]
# idx_design = np.where(df_all['fold'].isin(np.arange(0,8)))[0]
# idx_test   = np.where(df_all['fold'].isin([8,9]))[0]
# print(idx_train.shape)
# print(idx_design.shape)
# print(idx_test.shape)

# df_train = df_all_onehot.iloc[idx_train]
# df_val = df_all_onehot.iloc[idx_val]
# df_design = df_all_onehot.iloc[idx_design]
# df_test = df_all_onehot.iloc[idx_test]
# # df_train.head()


# #define independent variables, excluding those we
# vars_x = [var for var in df_all_onehot.columns.values if var not in (var_dep+vars_notToUse)]
# vars_x = vars_ind
# X_train = df_train[vars_x]
# X_val = df_val[vars_x]
# X_design = df_design[vars_x]
# X_test = df_test[vars_x]
# # X = df_all_onehot[vars_x]
# y = df_all_onehot['saleprice']




# from sklearn.preprocessing import StandardScaler
# standardScaler_ = StandardScaler()
# standardScaler_.fit(X_design)

# X_train = standardScaler_.transform(X_train)
# X_val   = standardScaler_.transform(X_val)
# X_design= standardScaler_.transform(X_design)
# X_test  = standardScaler_.transform(X_test)

In [252]:
#iterate over variables which we want to turn into a spline
#append slines to data

for var in vars_toUse: 
    df_ptiles, knots = fn_tosplines(var, df_all_onehot[var].values)
    #drop original variable after having created splines from it
    #as adding back the splines will also add back original var
    df_all_onehot.drop(columns = [var], inplace=True)
    #remove it from list of numeric independent vars for same reason
    vars_ind_numeric.remove(var)
    #append spline columns (for current var) to dataset
    df_all_onehot = pd.concat([df_all_onehot, df_ptiles],axis=1, sort=False)
    vars_ind_numeric.extend(df_ptiles.columns.tolist())

In [253]:
# df_temp.tail(10)

In [254]:
# df_all['misc_val'][df_all['misc_val']>0]

In [255]:
# df_all[['misc_val_{}'.format(i) for i in range(0,20)]].iloc[11]

In [256]:
#redefine independent variables
vars_ind = vars_ind_numeric+vars_ind_onehot
# vars_ind

In [257]:
srs_missing = df_temp.isnull().sum(axis=0) 
print(srs_missing[srs_missing>0]) #double check no more nans

Series([], dtype: int64)


In [258]:
# df_all.tail(10)

In [259]:
len(df_all)

2922

In [260]:
######## Recreate train and test data over df_all

In [261]:
### Separate train and test
idx_train  = np.where(df_all['fold'].isin(np.arange(0,6)))[0]
idx_val    = np.where(df_all['fold'].isin([6,7]))[0]
idx_design = np.where(df_all['fold'].isin(np.arange(0,8)))[0]
idx_test   = np.where(df_all['fold'].isin([8,9]))[0]
print(idx_train.shape)
print(idx_design.shape)
print(idx_test.shape)

df_train = df_all_onehot.iloc[idx_train]
df_val = df_all_onehot.iloc[idx_val]
df_design = df_all_onehot.iloc[idx_design]
df_test = df_all_onehot.iloc[idx_test]
# df_train.head()

(1749,)
(2335,)
(587,)


### Fit a penalised linear regression model using ElasticNetCV and/or ElasticNet from sklearn 

In [262]:
#define independent variables, excluding those we
vars_x = [var for var in df_all_onehot.columns.values if var not in (var_dep+vars_notToUse)]
vars_x = vars_ind
X_train = df_train[vars_x]
X_val = df_val[vars_x]
X_design = df_design[vars_x]
X_test = df_test[vars_x]
# X = df_all_onehot[vars_x]
y = df_all_onehot['saleprice']



## Standardise
from sklearn.preprocessing import StandardScaler
standardScaler_ = StandardScaler()
standardScaler_.fit(X_design)

X_train = standardScaler_.transform(X_train)
X_val   = standardScaler_.transform(X_val)
X_design= standardScaler_.transform(X_design)
X_test  = standardScaler_.transform(X_test)

l1_ratio = [.1, .5, .7, .9, .95, .99, 1]
model = ElasticNetCV(l1_ratio, alphas = [2**num for num in range(-10,10)])
model.fit(X=X_design, y=y[idx_design])

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


ElasticNetCV(alphas=[0.0009765625, 0.001953125, 0.00390625, 0.0078125, 0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512],
       copy_X=True, cv=None, eps=0.001, fit_intercept=True,
       l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000,
       n_alphas=100, n_jobs=1, normalize=False, positive=False,
       precompute='auto', random_state=None, selection='cyclic',
       tol=0.0001, verbose=0)

In [263]:
# X = df_train[vars_x]

In [264]:
fn_print_coefficients(model)

Learned polynomial for degree 639:
181056.8809 + 12031.6600x + 4735.0847x^2 + 1266.8365x^3 + -273.7478x^4 + 1403.0777x^5 + 1289.1912x^6 + -1013.8005x^7 + -2373.3916x^8 + 432.8170x^9 + 2650.9494x^10 + 2652.1411x^11 + 871.7250x^12 + -49.1066x^13 + -920.6167x^14 + 0.0000x^15 + 828.4454x^16 + 0.0000x^17 + 0.0000x^18 + 0.0000x^19 + 0.0000x^20 + 0.0000x^21 + 0.0000x^22 + 0.0000x^23 + 0.0000x^24 + 0.0000x^25 + 0.0000x^26 + 0.0000x^27 + 0.0000x^28 + 0.0000x^29 + 0.0000x^30 + 0.0000x^31 + 0.0000x^32 + -0.0000x^33 + -0.0000x^34 + 0.0000x^35 + 7425.5571x^36 + 98.9115x^37 + 0.0000x^38 + 0.0000x^39 + 0.0000x^40 + 0.0000x^41 + 0.0000x^42 + 0.0000x^43 + 0.0000x^44 + 0.0000x^45 + 0.0000x^46 + 0.0000x^47 + 0.0000x^48 + 0.0000x^49 + 0.0000x^50 + -0.0000x^51 + -0.0000x^52 + -0.0000x^53 + -0.0000x^54 + -1715.8861x^55 + 0.0000x^56 + 5882.0453x^57 + 0.0000x^58 + 0.0000x^59 + 0.0000x^60 + 0.0000x^61 + 0.0000x^62 + 0.0000x^63 + 0.0000x^64 + 0.0000x^65 + 0.0000x^66 + 2757.4763x^67 + 1289.0555x^68 + 0.0000x^69 

In [265]:
train_preds = model.predict(X_train)
val_preds = model.predict(X_val)
design_preds = model.predict(X_design)
test_preds = model.predict(X_test)

In [266]:
# mean absolute error in predictions

train_error = fn_MAE(y[idx_train], train_preds) 
validation_error = fn_MAE(y[idx_val], val_preds)
design_error = fn_MAE(y[idx_design], design_preds)


test_error = fn_MAE(y[idx_test], test_preds)

# on train data
print('train error', train_error)
print('\n')

# on validation data
print('validation naive (null) model', fn_MAE(np.median(y[idx_val]),  y[idx_val]))
print('validation error', validation_error)
print('\n')

# on design data
print('design (train + val) error', design_error)
print('\n')

# test data, null model
print('test naive (null) model', fn_MAE(np.median(y[idx_test]), y[idx_test]))
print('test error', test_error)

print('differnece between test and non-test error', test_error-design_error)

train error 12534.13450018554


validation naive (null) model 56596.921501706485
validation error 12303.690860095789


design (train + val) error 12476.301535263658


test naive (null) model 52206.85178875639
test error 13855.522603595698
differnece between test and non-test error 1379.2210683320409


In [267]:
# fig, axes = plt.subplots(3, 1, figsize=[12,24], sharex=True, sharey=True)
# axes[0].scatter(train_preds, y[idx_train], s = 1, alpha = 0.3)
# axes[0].set_title('Training data')
# axes[0].axes.xaxis.set_ticklabels([])
# axes[0].axes.yaxis.set_ticklabels([])
# axes[0].set_xlabel('predicted')
# axes[0].set_ylabel('actual')

# axes[1].scatter(val_preds, y[idx_val], s = 1)
# axes[1].set_title('Validation data')
# axes[1].set_xlabel('predicted')
# axes[1].set_ylabel('actual')

# axes[2].scatter(test_preds, y[idx_test], s = 1)
# axes[2].set_title('Test data')
# axes[2].set_xlabel('predicted')
# axes[2].set_ylabel('actual')



In [268]:
print(len(idx_train))
print(len(idx_val))
print(len(idx_design))

1749
586
2335


In [276]:
model_En = ElasticNet(alpha=480, l1_ratio=model.l1_ratio_, fit_intercept=True, normalize=False, max_iter=1000, copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=2018, selection='cyclic')
model_En.fit(X=X_design, y=y[idx_design])


train_preds = model_En.predict(X_train)
val_preds = model_En.predict(X_val)
design_preds = model_En.predict(X_design)
test_preds = model_En.predict(X_test)


# mean absolute error in predictions

train_error = fn_MAE(y[idx_train], train_preds) 
validation_error = fn_MAE(y[idx_val], val_preds)
design_error = fn_MAE(y[idx_design], design_preds)


test_error = fn_MAE(y[idx_test], test_preds)

# on train data
print('train error', train_error)
print('\n')

# on validation data
print('validation naive (null) model', fn_MAE(np.median(y[idx_val]),  y[idx_val]))
print('validation error', validation_error)
print('\n')

# on design data
print('design (train + val) error', design_error)
print('\n')

# test data, null model
print('test naive (null) model', fn_MAE(np.median(y[idx_test]), y[idx_test]))
print('test error', test_error)

print('differnece between test and non-test error', test_error-design_error)

train error 12967.816628738852


validation naive (null) model 56596.921501706485
validation error 12645.919320373698


design (train + val) error 12887.032122228366


test naive (null) model 52206.85178875639
test error 13979.621401755101
differnece between test and non-test error 1092.5892795267355


In [278]:
m_log_lambdas = -np.log10(model_En.alphas_)

font = {'size': 20}
plt.rc('font', **font)

fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(1, 1, 1)
ax1.plot(m_log_lambdas, model_En.mse_path_, ':')
ax1.plot(m_log_lambdas, model_En.mse_path_.mean(axis=-1),
         'k',
         label='Average across the folds',
         linewidth=2)

ax1.axvline(-np.log10(model_En.alpha_),
            linestyle='--', 
            color='k',
            label='alpha: CV estimate')

ax1.legend()

ax1.set_xlabel('-log10(lambda)')
ax1.set_ylabel('Mean square error')
ax1.set_title('Mean square error on each fold')
_ = ax1.axis('tight')

AttributeError: 'ElasticNet' object has no attribute 'alphas_'